In [1]:
from bs4 import BeautifulSoup
import requests
import json
import re

In [2]:
Website_name='Timber Lake Camp'
Website_url='https://www.timberlakecamp.com/'
HEADERS =({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
                   'Accept-Language':'en-US, en;q=0.5'})

home = requests.get(Website_url,headers=HEADERS)
home_soup = BeautifulSoup(home.content, 'lxml')

In [65]:
# Initialize an empty list to store descriptions
description = []

description_table = home_soup.find_all('div', attrs={'class': 'content'})


# Iterate through the description_table
for i in description_table:
    desc = i.find_all('p')  # Find all 'p' tags
    # Append the text from each <p> tag to the list if it's not empty
    for d in desc:
        text = d.text.strip()  # Stripping extra whitespace
        if text:  # Check if the text is not empty
            description.append(text)       

description

['Timber Lake Camp sits on a mountaintop surrounded by 500 acres in New York’s Catskill Mountains, just 2.5 hours from New York City. A traditional, co-ed camp, we feature two private, spring-fed lakes and an outstanding array of facilities on magnificently manicured grounds.',
 'At Timber Lake, we foster values of sharing, integrity, responsibility and respect in a friendly, safe environment of fun. The best camp experience gives children greater self-confidence and self-esteem. They learn to value others by being valued themselves. And they leave camp with memories that will last a lifetime. This is what we have been doing successfully for more than six decades.',
 'It is What Camp Was Always Meant To Be.']

In [69]:
logo_tag = home_soup.find('a', class_='logo').find('img')

# Check if data-lazy-src exists, otherwise fallback to src
if 'data-lazy-src' in logo_tag.attrs:
    logo_url = logo_tag['data-lazy-src']
else:
    logo_url = logo_tag['src']

print(logo_url)


https://www.timberlakecamp.com/wp-content/uploads/2015/12/Logo11.svg


In [58]:
# Initialize empty lists to store addresses and phone numbers
addresses = []
phone_numbers = []

# Extract location1 and location2 details
location1 = home_soup.find('section', attrs={'class': 'col-sm-3 col-sm-pull-6'}).text.strip()
location2 = home_soup.find('section', attrs={'class': 'right col-sm-3'}).text.strip()

# Combine both locations for processing
all_locations = location1 + "\n" + location2

# Process the text to extract addresses and phone numbers
lines = all_locations.split("\n")
current_address = ""

for line in lines:
    line = line.strip().replace('\xa0', ' ')  # Clean and normalize the line
    if line.isdigit() or (line.replace("-", "").isdigit() and len(line) >= 10):  # Check for phone numbers
        phone_numbers.append(line)
        if current_address:  # Append the current address when a phone number is found
            addresses.append(current_address.strip())
            current_address = ""
    elif line:  # If the line contains text (address or name)
        if not any(keyword in line.lower() for keyword in ["summer", "winter"]):  # Skip location names
            current_address += " " + line

# Ensure the last address is added if any
if current_address:
    addresses.append(current_address.strip())

# Define the mail and phone numbers (telephone numbers are already extracted)
mail = 'info@timberlakecamp.com'

# Define latitude and longitude values (can be updated with actual data)
lat = ['42.12123', '40.87364']
lon = ['-74.39387', '-73.62323']

# Create location_data dictionary with required fields
location_data = {
    "address": addresses,  # List of addresses
    "zip_code": [address.split()[-1] for address in addresses],  # Extract ZIP codes from addresses
    "latitude": lat,  # Latitude values
    "longitude": lon  # Longitude values
}

# Create contact_data dictionary with email and telephone numbers
contact_data = {
    "mail": mail,
    "telephone": phone_numbers  # List of telephone numbers
}

# Output the results
print(location_data)
print(contact_data)


{'address': ['354 Timber Lake Road Shandaken, NY 12480', '85 Crescent Beach Road Glen Cove, NY 11542'], 'zip_code': ['12480', '11542'], 'latitude': ['42.12123', '40.87364'], 'longitude': ['-74.39387', '-73.62323']}
{'mail': 'info@timberlakecamp.com', 'telephone': ['845-688-2266', '516-656-4200']}


In [59]:
# Initialize an empty list to store social media links
socials = []

# Find the section containing the social media links
social_section = home_soup.find('section', attrs={'class': 'col-sm-4 col-md-3 right socials'})

# Find all the <a> tags within the section
social_links = social_section.find_all('a', href=True)

# Iterate through the links and store them in the desired format
for link in social_links:
    href = link['href']
    if "facebook" in href:
        socials.append({'facebook': href})
    elif "twitter" in href:
        socials.append({'twitter': href})
    elif "instagram" in href:
        socials.append({'instagram': href})
    elif "youtube" in href:
        socials.append({'youtube': href})

# The socials list now contains the links in the desired format
print(socials)


[{'facebook': 'http://www.facebook.com/timberlakecamp'}, {'twitter': 'http://www.twitter.com/timberlakecamp'}, {'instagram': 'https://www.instagram.com/timberlakecamp/'}, {'youtube': 'https://www.youtube.com/user/TimberLakeCampVideos'}]


In [60]:
image_url = []
img_table=home_soup.find_all('div', class_='single-hero-slide')
img_table
for img in img_table:
    url = img.get('data-bg')
    image_url.append(url)

image_url

['https://www.timberlakecamp.com/wp-content/uploads/2015/11/video-poster.jpg',
 'https://www.timberlakecamp.com/wp-content/uploads/2015/11/tlc-gymnastics-aerial.jpg',
 'https://www.timberlakecamp.com/wp-content/uploads/2015/11/tlc-aerial-hero.jpg',
 'https://www.timberlakecamp.com/wp-content/uploads/2015/11/tlc-lax-hero.jpg',
 'https://www.timberlakecamp.com/wp-content/uploads/2015/11/tlc-drama-hero.jpg',
 'https://www.timberlakecamp.com/wp-content/uploads/2015/11/tlc-hero-boys-arms.jpg']

In [61]:
# URL and headers
rd_url = 'https://www.timberlakecamp.com/prospective-families/dates-fees-faqs/'
# Fetch and parse the page content
rd = requests.get(rd_url, headers=HEADERS)
rd_soup = BeautifulSoup(rd.content, 'lxml')

# Extract dates
dates_table = rd_soup.find('table', id='tablepress-19')
rows = dates_table.find_all('tr')[1:]  # Skip header row
start_date = rows[0].find_all('td')[1].text.strip()
end_date = rows[2].find_all('td')[1].text.strip()

# Extract fees and deposit
fees_table = rd_soup.find('table', id='tablepress-4')
fee = fees_table.find_all('td')[0].find('strong').text.strip()

deposit_table = fees_table.find_all('td')[1]
deposit_text = deposit_table.text.strip()
deposit = deposit_text.split('deposit on enrollment')[0].strip()  # Extract deposit amount

# Create a dictionary with extracted details
camp_details = {
    'name':'null',
    'start_date': start_date,
    'end_date': end_date,
    'cost': fee,
    'deposit': deposit,
    'eligible':'null',
    'duration':'null',
    'location':'null'
}


In [62]:
cat_url = 'https://www.timberlakecamp.com/prospective-families/program-activities/'

# Send GET request to fetch the main page
cat = requests.get(cat_url, headers=HEADERS)
cat_soup = BeautifulSoup(cat.content, 'lxml')

categories = []

# Locate all the relevant div elements
divs = cat_soup.find_all("div", class_="ezcol")

for div in divs:
    # Extract category name (from h3 or nested a tag)
    category_tag = div.find('h3')
    category = category_tag.get_text(strip=True) if category_tag else None

    # Extract image URL (handle lazy-loaded images)
    img_tag = div.find('img')
    image_url = img_tag.get('data-lazy-src') or img_tag.get('data-src') or img_tag.get('src') if img_tag else None

    # Extract the URL of the individual description page
    desc_url = div.find('a').get('href') if div.find('a') else None

    # Proceed if there is a valid URL for the description page
    if category and image_url and desc_url:
        # Check if the category is already in the list to avoid duplicates
        existing_category = next((item for item in categories if item['name'] == category), None)
        
        if existing_category:
            continue  # Skip if this category is already in the list
        
        # Scrape the description page if it's not already in the list
        desc = requests.get(desc_url, headers=HEADERS)
        desc_soup = BeautifulSoup(desc.content, 'lxml')

        # Extract the description from the description page
        heading = desc_soup.find('h1').get_text(strip=True) if desc_soup.find('h1') else 'No heading found'
        
        # Extract paragraph content (likely the description)
        paragraphs = desc_soup.find_all('p')
        description = ""
        for para in paragraphs:
            description += para.get_text(strip=True) + "\n"
        
        # Clean the description by removing unwanted phrases and whitespace
        description = description.replace("© Copyright Timber Lake Camp. All rights reserved.", "")
        description = description.replace("Website by", "")
        description = description.replace("We can’t wait to hear from you!", "")
        description = description.replace("Fields marked with*are required.", "")
        description = ' '.join(description.split())  # Remove excess whitespace

        # Add cleaned description to the category data
        category_data = {'name': category, 'description': description, 'image': image_url}
        categories.append(category_data)


In [63]:
act_url = 'https://www.timberlakecamp.com/prospective-families/program-activities/electives/'

a = requests.get(act_url, headers=HEADERS)
act_soup = BeautifulSoup(a.content, 'lxml')

act_divs = act_soup.find_all('div', attrs={'class': 'ezcol'})
Activities = []

# Loop through each div, extract the activities, and append them to the list
for div in act_divs:
    ul = div.find('ul')
    if ul:
        li_items = ul.find_all('li')
        for li in li_items:
            Activities.append({'name':li.get_text(),'description':'null','image':'null'})

In [70]:
Timber_lake= [{
    'name': Website_name,  
    'url': Website_url,
    'logo': logo_url,
    'description':description,
    'location': location_data,
    'contact': contact_data,
    'socials': socials,
    'image_urls':image_url,
    'sessions': camp_details,
    'categories': categories,
    'activities': Activities
}]
file_path = 'timber_lake.json'

# Export the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(Timber_lake, json_file, indent=4)